# Purpose
Series of steps to test for leakage in an existing project.

The example below is applied to the [Dirty Duck tutorial](https://github.com/dssg/dirtyduck), using City of Chicago inspections prioritization data. We assume that you have already followed the steps in the tutorial up to and including the Inspections section. The variables in ALL CAPS are things that need to be customized for each project, so make a copy of this notebook and personalize it for your project.

# Process



In [1]:
import sys
import os
import math
import warnings

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine

from sklearn import linear_model
from sklearn import metrics 
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from IPython.core.interactiveshell import InteractiveShell

from dd_credentials import * # credentials file containing database login variables (dbname, user, host, password, port)

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

In [5]:
def execute_sql(statement, dbname, user, host, password, port, isolation = False, results = True):
    """
    Use psycopg2 to execute PostgreSQL queries
    
    Input:
        statement (str): SQL statement to run in database
        dbname, user, host, password (str): database credentials
        isolation (bool): indicator for whether to change isolation level to autocommit; True for queries that cannot be run 
            from within a transation (see https://wiki.postgresql.org/wiki/Psycopg2_Tutorial), e.g. queries other than SELECT
        results (bool): indicator for whether the query is expected to output results;
            for example, True for SELECT statements and False for CREATE TABLE statements
    
    Output:
        relation (dataframe): query results or empty dataframe if results = False
    """
    conn = psycopg2.connect("dbname={} user={} host={} password={} port={}".format(dbname, user, host, password, port))
    cur = conn.cursor()
    if isolation:
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(statement)
    relation = pd.DataFrame()
    if results:
        results = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        relation = pd.DataFrame(results, columns=colnames)
    cur.close()
    conn.close()
    return relation


In [6]:
def randomize(df, do_not_randomize = None):
    """
    Randomize column values of a file. Each column is randomized independently.
    
    Inputs:
        df (dataframe): dataframe to randomize
        do_not_randomize (list): optional list of strings indicating names of 
            columns that should not be randomized
    Outputs:
        df (dataframe): dataframe of randomized data
    """     
    df_random = df.copy()
    if do_not_randomize:
        cols = [c for c in df.columns if c not in do_not_randomize]
    else:
        cols = df.columns
        
    for col in cols:
        #print('\t\tRandomizing column ' + col)
        df_random[col] = np.random.permutation(df_random[col])

    return df_random

In [7]:
# source: https://overlaid.net/2016/02/08/replace-words-in-files-or-strings-using-python/
def do_replacement(base_text, word_map):
    """
    Helper function for replace_words_in_file. Make replacements in base_text, as 
    indicated in  word_map.
    """
    for key, val in word_map.items():
        base_text = base_text.replace(key, val)
    return base_text

def replace_words_in_file(read_from, write_to, word_map):
    """
    Create copy of a file with certain words replaced
    
    Inputs
        read_from: name of file to read from
        write_to: name of new file to be created
        word_map: dictionary of mappings between words and their replacements
            (e.g. {'old_word': 'new_word'})
    Outputs
        None. Will create a new file with the name given in write_to
        
    """
    # Open your desired file as 't' and read the lines into string 'tempstr'
    t = open(read_from, 'r')
    tempstr = t.read()
    t.close()

    # Using the "replace_words" function, we'll pass in our tempstr to be used as the base, 
    # and our device_values to be used as replacement.  
    output = do_replacement(tempstr, word_map)

    # Write out the new config file
    fout = open(write_to, 'w')
    fout.write(output)
    fout.close()

In [8]:
def get_new_filename(filename, suffix):
    """
    Output a new filename (str) given a filename and suffix to append. Assumes file extension at the end is separated by a period.
    """
    k = filename.rfind(".")
    new_filename = filename[:k] + suffix + '.' + filename[k+1:]
    return new_filename

In [9]:
def generate_binary_pred(y_scores, k):
    """
    Generate binary predictions, where top k proportion of highest y_scores receive class of 1
    """
    cutoff = int(len(y_scores) * k)
    ind = np.argpartition(y_scores, -cutoff)[-cutoff:] # indices of top k probabilities
    y_pred = np.zeros(y_scores.shape[0])
    y_pred[ind] = 1
    return y_pred

## I. Create a randomized version of the raw schema

Pull the information schema from selected database.

In [6]:
# name of the database schema containing raw data to be randomized
RAW_SCHEMA = 'raw' 

In [7]:
statement = "SELECT * FROM information_schema.tables;"
tables = execute_sql(statement, dbname, user, host, password, port)

In [8]:
tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,food,inspections,active_facilities,BASE TABLE,None,None,None,None,None,YES,NO,None
1,food,postgis,geography_columns,VIEW,None,None,None,None,None,NO,NO,None
2,food,postgis,geometry_columns,VIEW,None,None,None,None,None,YES,NO,None
3,food,postgis,spatial_ref_sys,BASE TABLE,None,None,None,None,None,YES,NO,None
4,food,postgis,raster_columns,VIEW,None,None,None,None,None,NO,NO,None
5,food,postgis,raster_overviews,VIEW,None,None,None,None,None,NO,NO,None
6,food,cleaned,inspections,BASE TABLE,None,None,None,None,None,YES,NO,None
7,food,inspections,failed,BASE TABLE,None,None,None,None,None,YES,NO,None
8,food,inspections,failed_major_violation,BASE TABLE,None,None,None,None,None,YES,NO,None
9,food,pg_catalog,pg_statistic,BASE TABLE,None,None,None,None,None,YES,NO,None


Select tables from the raw schema; they will be randomized. In this tutorial, there is only one (raw.inspections).

In [9]:
raw = tables[tables.table_schema == RAW_SCHEMA]
raw.head()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
15,food,raw,inspections,BASE TABLE,None,None,None,None,None,YES,NO,None


First created the _randomized schema if it doesn't yet exist

In [10]:
statement = "CREATE SCHEMA IF NOT EXISTS {}_randomized;".format(RAW_SCHEMA)
output = execute_sql(statement, dbname, user, host, password, port, isolation=True, results = False)

Randomize every table in raw and write output to _randomized schema

In [11]:
for table_name in raw.table_name:
    print("Working on table {}".format(table_name))
    
    # Pull the table from original schema
    print("\tPulling table")
    statement = "SELECT * FROM {}.{};".format(RAW_SCHEMA, table_name)
    table = execute_sql(statement, dbname, user, host, password, port)
    
    # Randomize the table
    print("\tRandomizing")
    randomized_table = randomize(table)
    
    # Make a new table in _randomized schema
    print("\tUploading randomized version")
    statement = "CREATE TABLE IF NOT EXISTS {}_randomized.{} (LIKE {}.{});".format(RAW_SCHEMA, table_name, RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)
    
    # Write results into new table
    statement = "SELECT COUNT(*) FROM {}_randomized.{};".format(RAW_SCHEMA, table_name)
    output = execute_sql(statement, dbname, user, host, password, port)
    if output.iloc[0,0]>0: #  do nothing if new table already contains data
        print("\t*****SKIPPING TABLE {} -- it already has data".format(table_name))
    else:
        engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))
        randomized_table.to_sql(table_name, engine, schema = '{}_randomized'.format(RAW_SCHEMA), index = False, if_exists='append')
    

Working on table inspections
	Pulling table
	Randomizing
	Uploading randomized version


## II. Edit + execute preprocessing and config files so that they point to the newly randomized raw schema

New versions of the preprocessing and config files with an "_edited" suffix will be produced.

Make list of preprocessing files that get RAW_SCHEMA ready for triage experiments. The list should be in correct execution order, and file names should have some extension (e.g. .sql).

NOTE: The code below currently only works for .sql files. Need to add .py flexibility if necessary.

In [12]:
PREPROC_FILES = ['/home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table.sql',
                 '/home/ubuntu/dirtyduck/sql/create_violations_table.sql',
                 '/home/ubuntu/dirtyduck/sql/create_semantic_tables.sql',
                 '/home/ubuntu/dirtyduck/sql/create_inspections_schema.sql'
                ]

Also define the experiment config file name

In [13]:
EXP_CONFIG = "/home/ubuntu/dirtyduck/triage/experiment_config/inspections_test.yaml"

Make a dictionary of text replacements to apply across every preprocessing & config file. This should include:
- Names of schemas that are produced during preprocessing. Add a _randomized suffix to each of them.
- The "purpose" model group key from the experiment config file. In order to avoid overwriting existing model groups, the purpose must be changed to indicate that this experiment is for leakage detection.



In [14]:
WORD_MAP = {RAW_SCHEMA+'.': RAW_SCHEMA+'_randomized.', 
    'cleaned.': 'cleaned_randomized.', 
    'semantic.': 'semantic_randomized.',
    'inspections.': 'inspections_randomized.',
    "purpose: 'test'": "purpose: 'leakage-detection'"}

Apply find+replace to every preprocessing file.  

**NOTE**: Be sure to open the outputted files (with "_edited" suffix) to check that no unexpected replacements were made and that all necessary replacements are made. In this case, we have to manually edit some schema creation statements.

In [15]:
preproc_files_edited = []
for filename in PREPROC_FILES:
    new_filename = get_new_filename(filename, "_edited")
    replace_words_in_file(filename, new_filename, WORD_MAP)
    preproc_files_edited.append(new_filename)

Execute the edited preprocessing files

In [16]:
preproc_files_edited

['/home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_violations_table_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_semantic_tables_edited.sql',
 '/home/ubuntu/dirtyduck/sql/create_inspections_schema_edited.sql']

In [17]:
for filename in preproc_files_edited:
    print("Executing file ", filename)
    conn = psycopg2.connect("dbname={} user={} host={} password={} port={}".format(dbname, user, host, password, port))
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    with conn.cursor() as cursor:
        with open(filename, 'r') as f:
            cursor.execute(f.read())

Executing file  /home/ubuntu/dirtyduck/sql/create_cleaned_inspections_table_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_violations_table_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_semantic_tables_edited.sql
Executing file  /home/ubuntu/dirtyduck/sql/create_inspections_schema_edited.sql


Now edit the experiment config file in a similar fashion. Apply find+replace to config file.  

**NOTE**: Again, be sure to open the outputted file (with "_edited" suffix) to check that no unexpected replacements were made and that all necessary replacements are made.  
In this case, we increased test_durations and test_label_timespans from 1month to 3month to obtain a wider test window for more variation in predicted results. The same change was made to the original, non-randomized experiment as well.

In [19]:
new_exp_config = get_new_filename(EXP_CONFIG, '_edited')
replace_words_in_file(EXP_CONFIG, new_exp_config, WORD_MAP)

## III. Run an experiment with this new setup

Now that we've set up randomized versions of the original schemas in the database and created a new config file for a randomized experiment, it's time to run the actual triage experiment.

**NOTE**: This must be done from the command line!

- First validate the new config file. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml validate
- Then execute the experiment. For this tutorial: ./tutorial.sh triage --config_file inspections_test_edited.yaml run

In [24]:
# Tried running these commands within Jupyter but didn't work
#!!~/dirtyduck/tutorial.sh triage --config_file inspections_test_edited.yaml validate

['cat: .project-name: No such file or directory']

In [26]:
#!!~/dirtyduck/tutorial.sh triage --config_file inspections_test_edited.yaml run

['cat: .project-name: No such file or directory']

## IV. Align the results of randomized experiment against original experiment

First, we need to map the model groups from our new, randomized experiment to the model groups from the original experiment

Confirm that we now see a set of model groups where purpose = "leakage-detection"

In [53]:
statement = "SELECT * FROM results.model_groups;"
model_groups = execute_sql(statement, dbname, user, host, password, port)
model_groups

,model_group_id,model_type,model_parameters,feature_list,model_config
0,1,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
1,2,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
2,3,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
3,4,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
4,5,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
5,6,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'test', 'experiment_type': 'inspec..."
6,7,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
7,8,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 'sqrt'}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
8,9,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': None}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."
9,10,sklearn.tree.DecisionTreeClassifier,"{'max_depth': None, 'max_features': 1}",[inspections_entity_id_3month_type_canvass_sum...,"{'purpose': 'leakage-detection', 'experiment_t..."


Recall that a model group is defined by the classifier (model_type), its parameters (model_parameters), the features (feature_list), and the model_config. Since we varied the "purpose" key within model_config for these randomized experiments, all other properties of a model group should be used as index keys. This includes other model group keys in the model_config column.

In [54]:
# pull out all model group keys from model_config column and make new columns from them
list_of_lists = list(model_groups.model_config.apply(lambda x: list(x.keys())))
keys = set([key for x in list_of_lists for key in x])

for key in keys:
    model_groups[key] = model_groups.model_config.apply(lambda x: x.get(key, None))

In [55]:
# convert model_parameters and feature_list columns to strings so they can be used as index keys
model_groups['model_parameters'] = model_groups.model_parameters.apply(lambda x: ', '.join('{}{}'.format(key, val) for key, val in x.items()))
model_groups['feature_list'] = model_groups.feature_list.apply(lambda x: ', '.join(x))

In [56]:
index_cols = [col for col in model_groups.columns if col not in ['purpose', 'model_group_id', 'model_config']]
index_cols

['model_type',
 'model_parameters',
 'feature_list',
 'experiment_type',
 'label_definition']

In [57]:
model_groups = model_groups.set_index(index_cols)

Join the original and randomized model groups to create a wide (rather than long) version of model_group table. Each row of model_groups_wide corresponds to a particular model group configuration and indicates the model group ids of the original vs randomized versions.

In [58]:
# define original "purpose" model group key
ORIG_PURPOSE = 'test'

In [59]:
orig = model_groups[model_groups.purpose == ORIG_PURPOSE]
rand = model_groups[model_groups.purpose == 'leakage-detection']
model_groups_wide = pd.merge(orig[['model_group_id']], rand[['model_group_id']], how='right', left_index=True, right_index=True, suffixes = ['_orig', '_rand'])

In [60]:
model_groups_wide

model_group_id_orig  \
model_type                          model_parameters                feature_list                                       experiment_type            label_definition                        
sklearn.tree.DecisionTreeClassifier max_depth1, max_features1       inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              1   
                                    max_depth1, max_featuressqrt    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              2   
                                    max_depth1, max_featuresNone    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              3   
                                    max_depthNone, max_features1    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              4   
                                    max_depthNone, max_featuressqrt inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              5   
                                    max_depthNone, max_featuresNone inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              6   

                                                                                                                                                                    model_group_id_rand  
model_type                          model_parameters                feature_list                                       experiment_type            label_definition                       
sklearn.tree.DecisionTreeClassifier max_depth1, max_features1       inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              7  
                                    max_depth1, max_featuressqrt    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              8  
                                    max_depth1, max_featuresNone    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                              9  
                                    max_depthNone, max_features1    inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                             10  
                                    max_depthNone, max_featuressqrt inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                             11  
                                    max_depthNone, max_featuresNone inspections_entity_id_3month_type_canvass_sum, ... inspections prioritization failed                             12

Pick a model group configuration on which to test for leakage. Let's go with the first row, where model_group_id_orig = 1, model_group_id_rand = 7

In [61]:
# create variables for the model_group id's you selected
GROUP_ID_ORIG = 1
GROUP_ID_RAND = 7

Look at the models relevant to each model group

In [62]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(GROUP_ID_ORIG)
models_orig = execute_sql(statement, dbname, user, host, password, port)
models_orig

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,training_label_timespan
0,1,1,1b9a6f349b4b4558ac6c95a1a7f79155,2018-03-26 21:13:08.550605,2018-03-26 21:13:08.546242,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,f0dd57546dc8857d3a3b84ad85bec700,2015-08-13,False,fbca70a2027a22cdec76cd71182117ea,30 days
1,7,1,234dcf671ce54371e14ac231ef314e5f,2018-03-26 21:15:45.927328,2018-03-26 21:15:45.924006,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,f0dd57546dc8857d3a3b84ad85bec700,2016-08-13,False,05a3438b141af25314f980ce41f718a7,30 days


In [63]:
statement = "SELECT * FROM results.models where model_group_id = {};".format(GROUP_ID_RAND)
models_rand = execute_sql(statement, dbname, user, host, password, port)
models_rand

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,training_label_timespan
0,13,7,aca7525fa0a28e32d4e5c782b3cdf067,2018-03-26 21:20:34.924267,2018-03-26 21:20:34.919606,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,4c44030c15c7d20c6ed4fc7131b326a6,2015-08-13,False,04229ecfcc8961540b51394ab92b8919,30 days
1,19,7,e440fffe4903fe885b863d9fc0d9b650,2018-03-26 21:34:52.558446,2018-03-26 21:34:52.552622,sklearn.tree.DecisionTreeClassifier,"{'max_depth': 1, 'max_features': 1}",inspections_test,None,None,4c44030c15c7d20c6ed4fc7131b326a6,2016-08-13,False,0f068444c8667fd05f1f170a1fffb851,30 days


Pick a pair of comparable models on which to test for leakage, i.e. with matching train_end_time. Let's go with the first row of each table, i.e. model_id numbers 1 (original) and 13 (random).  
Grab the predictions made by each of these models.

In [3]:
# create variables for the model id's you selected
MODEL_ID_ORIG = 1
MODEL_ID_RAND = 13

In [14]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_ORIG)
predictions_orig = execute_sql(statement, dbname, user, host, password, port)
predictions_orig

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_timespan
0,1,1,2015-08-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
1,1,1,2015-09-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
2,1,1,2015-10-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
3,1,2,2015-08-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
4,1,2,2015-09-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
5,1,2,2015-10-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
6,1,5,2015-08-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
7,1,5,2015-09-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
8,1,5,2015-10-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days
9,1,6,2015-08-13,0.24082614798476037,NaN,None,None,e1423fcfda2de09bc169b49a723957e6,90 days


In [21]:
predictions_orig_sorted = predictions_orig.sort_values(by=['score', 'label_value'], ascending=[False, True])

In [13]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_RAND)
predictions_rand = execute_sql(statement, dbname, user, host, password, port)
predictions_rand

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_timespan
0,13,2,2015-08-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
1,13,2,2015-09-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
2,13,2,2015-10-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
3,13,3,2015-08-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
4,13,3,2015-09-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
5,13,3,2015-10-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
6,13,6,2015-08-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
7,13,6,2015-09-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
8,13,6,2015-10-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days
9,13,9,2015-08-13,0.1872791519434629,NaN,None,None,27a9331f6d2a75f12b1d3d886becaf69,90 days


In [24]:
predictions_rand_sorted = predictions_rand.sort_values(by=['score', 'label_value'], ascending=[False, True])

Grab the evaluation metrics for each of these models.

In [11]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_ORIG)
evaluations_orig = execute_sql(statement, dbname, user, host, password, port)
evaluations_orig

,model_id,evaluation_start_time,evaluation_end_time,as_of_date_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,1,2015-08-13,2015-10-13,30 days,precision@,5.0_pct,0.242,9626,500,2128,5
1,1,2015-08-13,2015-10-13,30 days,recall@,5.0_pct,0.0568609022556391,9626,500,2128,5
2,1,2015-08-13,2015-10-13,30 days,precision@,10.0_pct,0.23860329776915615,9626,1031,2128,5
3,1,2015-08-13,2015-10-13,30 days,recall@,10.0_pct,0.1156015037593985,9626,1031,2128,5
4,1,2015-08-13,2015-10-13,30 days,precision@,5_abs,0.5,9626,2,2128,5
5,1,2015-08-13,2015-10-13,30 days,recall@,5_abs,0.00046992481203007516,9626,2,2128,5
6,1,2015-08-13,2015-10-13,30 days,precision@,10_abs,0.3333333333333333,9626,3,2128,5
7,1,2015-08-13,2015-10-13,30 days,recall@,10_abs,0.00046992481203007516,9626,3,2128,5
8,1,2015-08-13,2015-10-13,30 days,precision@,25_abs,0.4,9626,5,2128,5
9,1,2015-08-13,2015-10-13,30 days,recall@,25_abs,0.0009398496240601503,9626,5,2128,5


In [12]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_RAND)
evaluations_rand = execute_sql(statement, dbname, user, host, password, port)
evaluations_rand

,model_id,evaluation_start_time,evaluation_end_time,as_of_date_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,13,2015-08-13,2015-10-13,30 days,precision@,5.0_pct,0.625,176,8,39,5
1,13,2015-08-13,2015-10-13,30 days,recall@,5.0_pct,0.1282051282051282,176,8,39,5
2,13,2015-08-13,2015-10-13,30 days,precision@,10.0_pct,0.4666666666666667,176,15,39,5
3,13,2015-08-13,2015-10-13,30 days,recall@,10.0_pct,0.1794871794871795,176,15,39,5
4,13,2015-08-13,2015-10-13,30 days,precision@,5_abs,0.0,176,0,39,5
5,13,2015-08-13,2015-10-13,30 days,recall@,5_abs,0.0,176,0,39,5
6,13,2015-08-13,2015-10-13,30 days,precision@,10_abs,0.0,176,0,39,5
7,13,2015-08-13,2015-10-13,30 days,recall@,10_abs,0.0,176,0,39,5
8,13,2015-08-13,2015-10-13,30 days,precision@,25_abs,0.0,176,0,39,5
9,13,2015-08-13,2015-10-13,30 days,recall@,25_abs,0.0,176,0,39,5


## V. Perform statistical tests on the randomized model to see if it performs better than random (and thus suggests data leakage)

Tutorial next steps:
- Bootstrap (?) the predicted scores to get a large sample of certain evaluation metrics.  
- Perform statistical tests for leakage on those evaluation metrics  

NOTE: 
- For simplicity, may skip bootstrapping and perform chi squared test on single result.
- Need to add complexity and variation to current Dirty Duck model; current one produces same scores for almost everyone.

Create bootstraped samples of the predicted scores

In [ ]:
# bootstrap_n (int): number of bootstrap samples to run
boostrap_n = 100
bootstrap_samples = []
for n in range(bootstrap_n):
    sample = predictions_rand.scores.sample(predictions_rand.shape[0], replace=True)
    bootstrap_samples.append(sample)

In [ ]:
bootstrap_samples

Generate labels at a specific threshold, e.g. 10% for each bootstrapped sample set

Calculate confusion matrix for each bootstrapped sample set

Perform chi-squared test 

In [ ]:
stats.chisquare(f_obs, f_exp, ddof=1)